In [1]:
import numpy as np # for Linear algebra
import pandas as pd # for data manipulation/CSV I/O
import matplotlib.pyplot as plt # for plotting Graphs
import seaborn as sns # for interactive graphs
pd.options.mode.chained_assignment = None    # To avoid the SettingWithCopyWarning

In [2]:
deliveries=pd.read_csv("dataset/deliveries.csv")
matches =pd.read_csv("dataset/matches.csv")

In [3]:
deliveries.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [4]:
print(deliveries.columns)

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'non_striker', 'bowler', 'is_super_over', 'wide_runs',
       'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed',
       'dismissal_kind', 'fielder'],
      dtype='object')


In [5]:
column_heads = list(set(list(deliveries.columns)).difference(set(['inning', 'non_striker', 'is_super_over', 'legbye_runs', 'bye_runs'])))

print(column_heads)

['match_id', 'player_dismissed', 'bowler', 'ball', 'batsman_runs', 'over', 'wide_runs', 'bowling_team', 'batting_team', 'penalty_runs', 'fielder', 'dismissal_kind', 'total_runs', 'noball_runs', 'batsman', 'extra_runs']


In [6]:
deliveries_bowlers = deliveries[column_heads]

print(deliveries_bowlers.head())

   match_id player_dismissed    bowler  ball  batsman_runs  over  wide_runs  \
0         1              NaN  TS Mills     1             0     1          0   
1         1              NaN  TS Mills     2             0     1          0   
2         1              NaN  TS Mills     3             4     1          0   
3         1              NaN  TS Mills     4             0     1          0   
4         1              NaN  TS Mills     5             0     1          2   

                  bowling_team         batting_team  penalty_runs fielder  \
0  Royal Challengers Bangalore  Sunrisers Hyderabad             0     NaN   
1  Royal Challengers Bangalore  Sunrisers Hyderabad             0     NaN   
2  Royal Challengers Bangalore  Sunrisers Hyderabad             0     NaN   
3  Royal Challengers Bangalore  Sunrisers Hyderabad             0     NaN   
4  Royal Challengers Bangalore  Sunrisers Hyderabad             0     NaN   

  dismissal_kind  total_runs  noball_runs    batsman  extra_ru

In [7]:
deliveries_bowlers['bowler'].nunique()

378

In [8]:
deliveries_bowlers.fillna("0", inplace=True)
deliveries_bowlers["Fours"] = np.where(deliveries_bowlers["batsman_runs"] == 4, 1, 0)
deliveries_bowlers["Sixes"] = np.where(deliveries_bowlers["batsman_runs"] == 6, 1, 0)

deliveries_bowlers['dismissal_kind'].unique()

array(['0', 'caught', 'bowled', 'run out', 'lbw', 'caught and bowled',
       'stumped', 'retired hurt', 'hit wicket', 'obstructing the field'],
      dtype=object)

In [9]:
dismissal_kind = ['caught', 'bowled', 'lbw', 'stumped', 'caught and bowled', 'hit wicket']
deliveries_bowlers["Wickets"] = np.where(deliveries_bowlers['dismissal_kind'].isin(dismissal_kind), 1, 0)

print(deliveries_bowlers.sample())

       match_id player_dismissed    bowler  ball  batsman_runs  over  \
24889       107                0  A Kumble     4             0    14   

       wide_runs                 bowling_team         batting_team  \
24889          0  Royal Challengers Bangalore  Chennai Super Kings   

       penalty_runs fielder dismissal_kind  total_runs  noball_runs   batsman  \
24889             0       0              0           1            0  SK Raina   

       extra_runs  Fours  Sixes  Wickets  
24889           1      0      0        0  


In [10]:
bowlers_info = deliveries_bowlers.groupby(['bowler', 'match_id'], as_index=False).agg(
    {'over': lambda x: x.nunique(), 'total_runs': 'sum', 'Fours': 'sum', 'Sixes': 'sum', 'Wickets': 'sum'}).reset_index()

print(bowlers_info.sample())

      index       bowler  match_id  over  total_runs  Fours  Sixes  Wickets
1206   1206  BCJ Cutting      7920     1          15      1      1        0


In [11]:
bowlers_info['3WI'] = np.where(bowlers_info['Wickets']>2, 1, 0)
career_stats = bowlers_info.groupby('bowler', as_index=False).agg(
    {'match_id': 'count', 'total_runs': 'sum', 'over': 'sum', 'Fours': 'sum', 'Sixes': 'sum', 
     'Wickets': 'sum', '3WI': 'sum'}).reset_index()

most_wickets_in_match = bowlers_info.groupby(['bowler']).agg({'Wickets': 'max'}).reset_index()
bowlers_info = pd.merge(bowlers_info, most_wickets_in_match, on=['bowler', 'Wickets'])

least_runs_for_most_wickets = bowlers_info.groupby(['bowler']).agg({'total_runs': 'min'}).reset_index()
bowlers_info = pd.merge(bowlers_info, least_runs_for_most_wickets, on=['bowler', 'total_runs'])

print(bowlers_info.head())

   index          bowler  match_id  over  total_runs  Fours  Sixes  Wickets  \
0      8  A Ashish Reddy       376     4          25      1      1        3   
1     21      A Chandila       365     4          13      1      0        4   
2     35     A Choudhary        42     4          17      1      0        2   
3     37     A Dananjaya      7902     4          48      2      4        0   
4     39      A Flintoff       122     3          11      1      0        1   

   3WI  
0    1  
1    1  
2    0  
3    0  
4    0  


In [12]:
bowlers_info['BBI'] = ( "="+ '\"' + bowlers_info['Wickets'].map(str)) + '/' + (bowlers_info['total_runs'].map(str) + '\"')

print(bowlers_info.head())

   index          bowler  match_id  over  total_runs  Fours  Sixes  Wickets  \
0      8  A Ashish Reddy       376     4          25      1      1        3   
1     21      A Chandila       365     4          13      1      0        4   
2     35     A Choudhary        42     4          17      1      0        2   
3     37     A Dananjaya      7902     4          48      2      4        0   
4     39      A Flintoff       122     3          11      1      0        1   

   3WI      BBI  
0    1  ="3/25"  
1    1  ="4/13"  
2    0  ="2/17"  
3    0  ="0/48"  
4    0  ="1/11"  


In [13]:
bowlers_info['bowler'].count()

384

In [14]:
bowlers_info[bowlers_info.duplicated(['bowler'], keep='first')]

,index,bowler,match_id,over,total_runs,Fours,Sixes,Wickets,3WI,BBI
161,3410,K Rabada,41,4,28,2,1,2,0,"=""2/28"""
229,4679,N Rana,7908,2,11,1,0,2,0,"=""2/11"""
248,5002,PJ Cummins,55,4,22,1,0,2,0,"=""2/22"""
287,6195,Rashid Khan,7918,4,19,2,0,3,1,"=""3/19"""
288,6204,Rashid Khan,7952,4,19,2,0,3,1,"=""3/19"""
316,6605,SE Bond,230,4,24,2,0,2,0,"=""2/24"""


In [15]:
bowlers_info[bowlers_info.duplicated(['bowler'], keep='last')]

,index,bowler,match_id,over,total_runs,Fours,Sixes,Wickets,3WI,BBI
160,3407,K Rabada,31,4,28,3,0,2,0,"=""2/28"""
228,4678,N Rana,7896,1,11,1,1,2,0,"=""2/11"""
247,4996,PJ Cummins,24,4,22,1,0,2,0,"=""2/22"""
286,6176,Rashid Khan,6,4,19,1,0,3,1,"=""3/19"""
287,6195,Rashid Khan,7918,4,19,2,0,3,1,"=""3/19"""
315,6601,SE Bond,196,4,24,2,1,2,0,"=""2/24"""


In [16]:
bowlers_info = bowlers_info.drop_duplicates('bowler', keep='last')
career_stats = pd.merge(career_stats, bowlers_info[['bowler', 'BBI']], on='bowler')

print(career_stats.head())

   index          bowler  match_id  total_runs  over  Fours  Sixes  Wickets  \
0      0  A Ashish Reddy        20         400    45     26     20       18   
1      1      A Chandila        12         245    39     17     10       11   
2      2     A Choudhary         5         144    17     13      6        5   
3      3     A Dananjaya         1          48     4      2      4        0   
4      4      A Flintoff         3         106    11      7      7        2   

   3WI      BBI  
0    1  ="3/25"  
1    1  ="4/13"  
2    0  ="2/17"  
3    0  ="0/48"  
4    0  ="1/11"  


In [17]:
career_stats = career_stats.rename(columns={'bowler': 'Bowler', 'total_runs': 'Runs', 'match_id': 'Matches', 'over': 'Overs'})
career_stats['Average'] = np.around(career_stats['Runs'] / career_stats['Wickets'], 2)
career_stats['Economy'] = np.around(career_stats['Runs'] / career_stats['Overs'], 2)
career_stats['SR'] = np.around((career_stats['Overs']*6) / career_stats['Wickets'], 2)

# Reordering columns for presentation
sequence = ['Bowler', 'Matches', 'Wickets', 'Average', 'Economy', 'SR', 'BBI',  '3WI', 'Fours', 'Sixes']

career_stats = career_stats[sequence]


In [22]:
career_stats.sort_values(by='Wickets', ascending=False).reset_index(drop='T')[:10]


,Bowler,Matches,Wickets,Average,Economy,SR,BBI,3WI,Fours,Sixes
0,SL Malinga,110,154,19.70,7.02,16.83,"=""5/13""",15,277,63
1,A Mishra,136,146,24.49,7.47,19.68,"=""5/17""",15,204,160
2,PP Chawla,143,139,26.92,7.80,20.72,"=""4/21""",11,273,150
3,DJ Bravo,119,136,24.82,8.54,17.43,"=""4/22""",10,261,129
4,Harbhajan Singh,146,134,27.80,7.18,23.24,"=""5/18""",9,268,127
5,B Kumar,102,120,23.16,7.31,19.00,"=""5/20""",12,274,71
6,SP Narine,97,112,22.94,6.69,20.57,"=""5/20""",12,194,79
7,UT Yadav,107,111,29.27,8.60,20.43,"=""4/24""",13,344,97
8,R Ashwin,122,110,27.05,6.85,23.67,"=""4/34""",5,177,106
9,A Nehra,88,106,23.93,7.93,18.11,"=""4/11""",14,279,76


In [19]:
export_csv = career_stats.to_csv ('dataset/bowlers.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path